In [1]:
from ase.io import read,write 
from mace.calculators.mace import MACECalculator
import matplotlib.pyplot as plt
import numpy as np 
import os, glob

from ase.filters import FrechetCellFilter
from ase.optimize import FIRE

/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = torch.lo

# ToDo :

Need to instead have the Vacancy and NEB take the optimized structures from MACE as it's inputs, currently using VASP inputs. 

This requires me to do the following: 
1) relax the initial perfect structures with MACE
2) read the number in the vacancy files to figure out which index to remove
3) relax the vacancy structures with MACE 
4) use the relaxed vacancy structures as the input for the NEB calculations 
5) Re-run the NEB calculations with the MACE optimized structures as the input 
6) Compare the VASP and MACE NEB calculations 


## Helper functions

In [29]:
def compare_atoms(atoms1, atoms2, verbose=False):
    """Compare the postitions, energies, forces, and stresses of two atoms objects. 
    Inputs:
        atoms1: ase.Atoms object, the atoms object relaxed with VASP
        atoms2: ase.Atoms object, the atoms object relaxed with MACE
        verbose: bool, if True, print the results of the comparison
    Returns:
        rmsd: float, the root mean square deviation of the positions
        rmse_energy: float, the root mean square error of the energies
        rmse_forces: float, the root mean square error of the forces
        rmse_stress: float, the root mean square error of the stresses"""
    # get positions of atoms1 and atoms2
    pos1 = atoms1.get_positions()
    pos2 = atoms2.get_positions()

    # compute the RMSD between the positions of the atoms objects
    rmsd = np.sqrt(np.mean((pos1 - pos2)**2))

    # get the energies of the atoms objects
    energy1 = atoms1.get_potential_energy()/len(atoms1)
    energy2 = atoms2.get_potential_energy()/len(atoms2)

    # compute the RMSE of the energies
    rmse_energy = np.sqrt(np.mean((energy1 - energy2)**2))
    if verbose:
        print(f"Atoms 1 energy: {energy1} eV/atom, Atoms 2 energy: {energy2} eV/atom, RMSE Energy: {rmse_energy} eV/atom")

    # get the forces of the atoms objects
    forces1 = atoms1.get_forces()
    forces2 = atoms2.get_forces()
    # compute the RMSE of the forces
    rmse_forces = np.sqrt(np.mean((forces1 - forces2)**2))
    #if verbose:
        #print(f"Atoms 1 forces: {forces1}, Atoms 2 forces: {forces2}, RMSE Forces: {rmse_forces}")

    # get the stresses of the atoms objects
    stress1 = atoms1.get_stress(voigt=False)
    stress2 = atoms2.get_stress(voigt=False)
    # compute the RMSE of the stresses
    rmse_stress = np.sqrt(np.mean((stress1 - stress2)**2))
    #if verbose:
        #print(f"Atoms 1 stresses: {stress1}, Atoms 2 stresses: {stress2}, RMSE Stress: {rmse_stress}")

    return rmsd, rmse_energy, rmse_forces, rmse_stress

# Compare perfect systems between VASP and MACE

In [51]:
# perfect data 
perf_path = '../data/post_mrs_static/mrs_perf_static'

# get the paths for every OUTCAR file present in the perf_path, needs to search recursively
outcar_paths = glob.glob(os.path.join(perf_path, '**', 'OUTCAR'), recursive=True)

perf_atoms = {}
for outcar_path in outcar_paths:
    atoms = read(outcar_path)
    # get the name of the final subdirectory
    subdir_name = os.path.basename(os.path.dirname(outcar_path))
    perf_atoms[subdir_name] = atoms

print(perf_atoms.keys())

dict_keys(['Cr5Ti5V115', 'Cr3Ti5V115W3Zr2', 'Cr4Ti7V111W4Zr2', 'Cr2Ti2V120W2Zr2', 'Cr5Ti9V107W4Zr3', 'Cr6Ti11V102W6Zr3'])


In [52]:
model_path = '../potentials/gen_6_model_0_L0_isolated-2026-01-16-finetuned_fp64_nh10000_lr1e-4_stagetwo.model'
calc = MACECalculator(model_path=[model_path], device='cuda', use_cueq=True, default_dtype='float32')


def relax_atoms(atoms, calc, relax_cell=False, fmax=0.01):
    """Relax the atoms with the MACE calculator"""
    new_atoms = atoms.copy()
    new_atoms.calc = calc
    if relax_cell:
        fcf = FrechetCellFilter(new_atoms)
        opt = FIRE(fcf)
    else:
        opt = FIRE(new_atoms)
    opt.run(steps=100, fmax=fmax)

    return new_atoms



Default dtype float32 does not match model dtype float64, converting models to float32.


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

In [53]:
# get the initial_structures of all the perfect atoms 

initial_path = '../data/pre_mrs_comparison/perf_t2/'

# Find all subdirectories within initial_path
subdir_paths = glob.glob(os.path.join(initial_path, '*'))
poscar_atoms = {}

for subdir_path in subdir_paths:
    if not os.path.isdir(subdir_path):
        continue  # Skip if not a directory

    subdir_name = os.path.basename(subdir_path)
    #poscar_0_path = os.path.join(subdir_path, 'POSCAR-0')
    poscar_path = os.path.join(subdir_path, 'POSCAR')

    #if os.path.exists(poscar_0_path):
        #atoms = read(poscar_0_path)
        #print(f"Reading POSCAR-0 from: {subdir_path}")
    #elif os.path.exists(poscar_path):
    print(f"Reading POSCAR from: {subdir_path}")
    atoms = read(poscar_path)
    #else:
        #print(f"No POSCAR or POSCAR-0 found in: {subdir_path}")
        #continue # Skip to the next subdirectory if no POSCAR file is found

    poscar_atoms[subdir_name] = atoms # Store the atoms object, keyed by subdirectory name

print("Atoms objects read from POSCAR files:")
for name in poscar_atoms:
    print(name)

Reading POSCAR from: ../data/pre_mrs_comparison/perf_t2/Cr5Ti5V115
Reading POSCAR from: ../data/pre_mrs_comparison/perf_t2/Cr3Ti5V115W3Zr2
Reading POSCAR from: ../data/pre_mrs_comparison/perf_t2/Cr4Ti7V111W4Zr2
Reading POSCAR from: ../data/pre_mrs_comparison/perf_t2/Cr2Ti2V120W2Zr2
Reading POSCAR from: ../data/pre_mrs_comparison/perf_t2/Cr5Ti9V107W4Zr3
Reading POSCAR from: ../data/pre_mrs_comparison/perf_t2/Cr6Ti11V102W6Zr3
Atoms objects read from POSCAR files:
Cr5Ti5V115
Cr3Ti5V115W3Zr2
Cr4Ti7V111W4Zr2
Cr2Ti2V120W2Zr2
Cr5Ti9V107W4Zr3
Cr6Ti11V102W6Zr3


In [54]:
# relax each of the initial structures with MACE calculator 
relaxed_atoms = {}
for name, atoms in poscar_atoms.items():
    relaxed_atoms[name] = relax_atoms(atoms, calc, relax_cell=True)

      Step     Time          Energy          fmax
FIRE:    0 22:14:38    -1119.121460        0.302429
FIRE:    1 22:14:38    -1119.126465        0.301417
FIRE:    2 22:14:38    -1119.136719        0.299413
FIRE:    3 22:14:38    -1119.150879        0.296445
FIRE:    4 22:14:38    -1119.168213        0.292526
FIRE:    5 22:14:38    -1119.188232        0.287622
FIRE:    6 22:14:38    -1119.208862        0.281646
FIRE:    7 22:14:39    -1119.229248        0.274463
FIRE:    8 22:14:39    -1119.252075        0.264998
FIRE:    9 22:14:39    -1119.277466        0.252682
FIRE:   10 22:14:39    -1119.306396        0.237027
FIRE:   11 22:14:39    -1119.339355        0.217682
FIRE:   12 22:14:39    -1119.375366        0.194442
FIRE:   13 22:14:39    -1119.413086        0.167126
FIRE:   14 22:14:39    -1119.447266        0.135337
FIRE:   15 22:14:39    -1119.475464        0.124445
FIRE:   16 22:14:39    -1119.499756        0.100309
FIRE:   17 22:14:39    -1119.510010        0.068615
FIRE:   18 22:

In [10]:
scratch_model_path = '../potentials/gen_6_model_0_L0_isolated-2026-01-16_stagetwo.model'
scratch_calc = MACECalculator(model_path=[scratch_model_path], device='cuda', default_dtype='float32')
scratch_relaxed_atoms = {}
for name, atoms in poscar_atoms.items():
    scratch_relaxed_atoms[name] = relax_atoms(atoms, scratch_calc, relax_cell=True)


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

      Step     Time          Energy          fmax
FIRE:    0 20:57:34    -1119.859863        0.150476
FIRE:    1 20:57:35    -1119.861084        0.139785
FIRE:    2 20:57:35    -1119.863037        0.119559
FIRE:    3 20:57:35    -1119.865479        0.091996
FIRE:    4 20:57:35    -1119.867676        0.082678
FIRE:    5 20:57:35    -1119.869751        0.079761
FIRE:    6 20:57:35    -1119.871460        0.076482
FIRE:    7 20:57:35    -1119.873291        0.072962
FIRE:    8 20:57:35    -1119.874512        0.068932
FIRE:    9 20:57:35    -1119.876221        0.064408
FIRE:   10 20:57:35    -1119.878540        0.059374
FIRE:   11 20:57:35    -1119.880127        0.053766
FIRE:   12 20:57:35    -1119.882690        0.047453
FIRE:   13 20:57:35    -1119.885254        0.040271
FIRE:   14 20:57:35    -1119.887573        0.032072
FIRE:   15 20:57:35    -1119.889160        0.026161
FIRE:   16 20:57:35    -1119.890259        0.030224
FIRE:   17 20:57:35    -1119.891113        0.032513
FIRE:   18 20:

In [8]:
# compare the vasp and the mace relaxed atoms 
atoms_names = relaxed_atoms.keys()

for atoms_name in atoms_names:
    vasp_atoms = perf_atoms[atoms_name]
    mace_atoms = relaxed_atoms[atoms_name]
    rmsd, rmse_energy, rmse_forces, rmse_stress = compare_atoms(vasp_atoms, mace_atoms, verbose=True)
    print(f"{atoms_name} - RMSD: {rmsd}, RMSE Energy: {rmse_energy}, RMSE Forces: {rmse_forces}, RMSE Stress: {rmse_stress}")


Atoms 1 energy: -1115.99073208, Atoms 2 energy: -1119.516357421875, RMSE Energy: 3.5256253418749566
Cr5Ti5V115 - RMSD: 3.716776768041814, RMSE Energy: 3.5256253418749566, RMSE Forces: 0.0017083731917243124, RMSE Stress: 0.00031863046763553227
Atoms 1 energy: -1151.08037927, Atoms 2 energy: -1154.847412109375, RMSE Energy: 3.767032839374906
Cr3Ti5V115W3Zr2 - RMSD: 3.54039004334665, RMSE Energy: 3.767032839374906, RMSE Forces: 0.0020174330759882976, RMSE Stress: 0.00011956827055416192
Atoms 1 energy: -1153.17243036, Atoms 2 energy: -1156.93701171875, RMSE Energy: 3.7645813587500925
Cr4Ti7V111W4Zr2 - RMSD: 2.6913537102466116, RMSE Energy: 3.7645813587500925, RMSE Forces: 0.0021586451447117233, RMSE Stress: 0.0002612507436508987
Atoms 1 energy: -1150.42916965, Atoms 2 energy: -1154.0670166015625, RMSE Energy: 3.637846951562551
Cr2Ti2V120W2Zr2 - RMSD: 2.5360889695984583, RMSE Energy: 3.637846951562551, RMSE Forces: 0.002025767699628888, RMSE Stress: 0.00023447368938375659
Atoms 1 energy: -1

In [9]:
# compare the vasp and the mace relaxed atoms 
atoms_names = relaxed_atoms.keys()

for atoms_name in atoms_names:
    vasp_atoms = perf_atoms[atoms_name]
    mace_atoms = scratch_relaxed_atoms[atoms_name]
    rmsd, rmse_energy, rmse_forces, rmse_stress = compare_atoms(vasp_atoms, mace_atoms, verbose=True)
    print(f"{atoms_name} - RMSD: {rmsd}, RMSE Energy: {rmse_energy}, RMSE Forces: {rmse_forces}, RMSE Stress: {rmse_stress}")

Atoms 1 energy: -1115.99073208, Atoms 2 energy: -1119.891357421875, RMSE Energy: 3.9006253418749566
Cr5Ti5V115 - RMSD: 3.707191136298724, RMSE Energy: 3.9006253418749566, RMSE Forces: 0.0028561809977911065, RMSE Stress: 0.00015259480713590435
Atoms 1 energy: -1151.08037927, Atoms 2 energy: -1155.172607421875, RMSE Energy: 4.092228151874906
Cr3Ti5V115W3Zr2 - RMSD: 3.539013423351061, RMSE Energy: 4.092228151874906, RMSE Forces: 0.0016145842932372809, RMSE Stress: 0.0001495788420197441
Atoms 1 energy: -1153.17243036, Atoms 2 energy: -1157.08642578125, RMSE Energy: 3.9139954212500925
Cr4Ti7V111W4Zr2 - RMSD: 2.6903559939039434, RMSE Energy: 3.9139954212500925, RMSE Forces: 0.001412498595409336, RMSE Stress: 0.0002559329129996279
Atoms 1 energy: -1150.42916965, Atoms 2 energy: -1154.603759765625, RMSE Energy: 4.174590115625051
Cr2Ti2V120W2Zr2 - RMSD: 2.535661204005909, RMSE Energy: 4.174590115625051, RMSE Forces: 0.0008721446469171035, RMSE Stress: 0.00023937131635705107
Atoms 1 energy: -114

# Working with vacancies

## From VASP starting point

In [17]:
# vacancy data
vacancy_path = '../data/post_mrs_static/mrs_vac_static'

# get the paths for every OUTCAR file present in the vacancy_path, needs to search recursively
outcar_paths = glob.glob(os.path.join(vacancy_path, '**', 'OUTCAR'), recursive=True)

vac_atoms = {}
for outcar_path in outcar_paths:
    atoms = read(outcar_path)
    # get the name of the final subdirectory
    path_parts = outcar_path.split(os.sep)
    subdir_name = f"{path_parts[-3]}_{path_parts[-2]}"
    vac_atoms[subdir_name] = atoms

print(vac_atoms.keys())


dict_keys(['Cr5Ti5V115_Rel_Start_17', 'Cr5Ti5V115_Rel_End_23', 'Cr3Ti5V115W3Zr2_End_Index_66', 'Cr3Ti5V115W3Zr2_Start_Index_60', 'Cr4Ti7V111W4Zr2_Start_Index_14', 'Cr4Ti7V111W4Zr2_End_Index_15', 'Cr2Ti2V120W2Zr2_Start_Index_98', 'Cr2Ti2V120W2Zr2_End_Index_117', 'Cr5Ti9V107W4Zr3_Start_Index_34', 'Cr5Ti9V107W4Zr3_End_Index_110', 'Cr6Ti11V102W6Zr3_End_Index_54', 'Cr6Ti11V102W6Zr3_Start_Index_60'])


In [35]:
model_path = '../potentials/gen_6_model_0_L0_isolated-2026-01-16-finetuned_fp64_nh10000_lr1e-4_stagetwo.model'
#model_path = '../potentials/gen_5_model_0-11-28_stagetwo.model'
calc = MACECalculator(model_path=[model_path], device='cuda', use_cueq=True, default_dtype='float32')


def relax_atoms(atoms, calc, relax_cell=False, fmax=0.01):
    """Relax the atoms with the MACE calculator"""
    new_atoms = atoms.copy()
    new_atoms.calc = calc
    if relax_cell:
        fcf = FrechetCellFilter(new_atoms)
        opt = FIRE(fcf)
    else:
        opt = FIRE(new_atoms)
    opt.run(steps=100, fmax=fmax)

    return new_atoms

/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

Default dtype float32 does not match model dtype float64, converting models to float32.


In [36]:
in_vac_path = '../data/pre_mrs_comparison/vac/'

# Find all subdirectories recursively within initial_path
subdir_paths = glob.glob(os.path.join(in_vac_path, '**/POSCAR'), recursive=True)
vac_poscar_atoms = {}

for poscar_path in subdir_paths:
    # Get the directory containing POSCAR
    subdir_path = os.path.dirname(poscar_path)
    
    # Get both the composition and index directories
    comp_dir = os.path.basename(os.path.dirname(subdir_path))  # e.g., "Cr4Ti7V111W4Zr2"
    index_dir = os.path.basename(subdir_path)  # e.g., "Start_Index_14"
    
    # Create a meaningful key for the dictionary
    subdir_name = f"{comp_dir}_{index_dir}"
    
    atoms = read(poscar_path)
    print(f"Reading POSCAR from: {subdir_path}")
    vac_poscar_atoms[subdir_name] = atoms

print("\nAtoms objects read from POSCAR files:")
for name in vac_poscar_atoms:
    print(name)

Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr5Ti5V115/Rel_Start_17
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr5Ti5V115/Rel_End_23
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr3Ti5V115W3Zr2/End_Index_66
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr3Ti5V115W3Zr2/Start_Index_60
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr4Ti7V111W4Zr2/Start_Index_14
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr4Ti7V111W4Zr2/End_Index_15
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr2Ti2V120W2Zr2/Start_Index_98
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr2Ti2V120W2Zr2/End_Index_117
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr5Ti9V107W4Zr3/Start_Index_34
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr5Ti9V107W4Zr3/End_Index_110
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr6Ti11V102W6Zr3/End_Index_54
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr6Ti11V102W6Zr3/Start_Index_60

Atoms objects read from P

In [37]:
print(vac_atoms.keys())
print(vac_poscar_atoms.keys())

dict_keys(['Cr5Ti5V115_Rel_Start_17', 'Cr5Ti5V115_Rel_End_23', 'Cr3Ti5V115W3Zr2_End_Index_66', 'Cr3Ti5V115W3Zr2_Start_Index_60', 'Cr4Ti7V111W4Zr2_Start_Index_14', 'Cr4Ti7V111W4Zr2_End_Index_15', 'Cr2Ti2V120W2Zr2_Start_Index_98', 'Cr2Ti2V120W2Zr2_End_Index_117', 'Cr5Ti9V107W4Zr3_Start_Index_34', 'Cr5Ti9V107W4Zr3_End_Index_110', 'Cr6Ti11V102W6Zr3_End_Index_54', 'Cr6Ti11V102W6Zr3_Start_Index_60'])
dict_keys(['Cr5Ti5V115_Rel_Start_17', 'Cr5Ti5V115_Rel_End_23', 'Cr3Ti5V115W3Zr2_End_Index_66', 'Cr3Ti5V115W3Zr2_Start_Index_60', 'Cr4Ti7V111W4Zr2_Start_Index_14', 'Cr4Ti7V111W4Zr2_End_Index_15', 'Cr2Ti2V120W2Zr2_Start_Index_98', 'Cr2Ti2V120W2Zr2_End_Index_117', 'Cr5Ti9V107W4Zr3_Start_Index_34', 'Cr5Ti9V107W4Zr3_End_Index_110', 'Cr6Ti11V102W6Zr3_End_Index_54', 'Cr6Ti11V102W6Zr3_Start_Index_60'])


In [38]:
# relax each of the initial structures with MACE calculator 
relaxed_vac_atoms = {}
for name, atoms in vac_poscar_atoms.items():
    relaxed_vac_atoms[name] = relax_atoms(atoms, calc, relax_cell=False)



      Step     Time          Energy          fmax
FIRE:    0 21:35:19    -1107.939941        0.464739
FIRE:    1 21:35:19    -1107.963379        0.431993
FIRE:    2 21:35:19    -1108.005127        0.373133
FIRE:    3 21:35:19    -1108.059204        0.352674
FIRE:    4 21:35:20    -1108.117310        0.339742
FIRE:    5 21:35:20    -1108.173218        0.324434
FIRE:    6 21:35:20    -1108.220459        0.304925
FIRE:    7 21:35:20    -1108.255493        0.277859
FIRE:    8 21:35:20    -1108.280762        0.236210
FIRE:    9 21:35:20    -1108.299438        0.220436
FIRE:   10 21:35:20    -1108.310059        0.237303
FIRE:   11 21:35:20    -1108.313110        0.225319
FIRE:   12 21:35:20    -1108.318359        0.202709
FIRE:   13 21:35:20    -1108.325073        0.171900
FIRE:   14 21:35:20    -1108.332031        0.135956
FIRE:   15 21:35:20    -1108.338379        0.097980
FIRE:   16 21:35:20    -1108.343506        0.076789
FIRE:   17 21:35:20    -1108.346802        0.072272
FIRE:   18 21:

In [39]:
# compare the vasp and the mace relaxed atoms 
atoms_names = relaxed_vac_atoms.keys()

for atoms_name in atoms_names:
    vasp_atoms = vac_atoms[atoms_name]
    mace_atoms = relaxed_vac_atoms[atoms_name]
    rmsd, rmse_energy, rmse_forces, rmse_stress = compare_atoms(vasp_atoms, mace_atoms, verbose=True)
    print(f"{atoms_name} - RMSD: {rmsd}, RMSE Energy: {rmse_energy}, RMSE Forces: {rmse_forces}, RMSE Stress: {rmse_stress}")

Atoms 1 energy: -8.907411827741935 eV/atom, Atoms 2 energy: -8.938336772303428 eV/atom, RMSE Energy: 0.03092494456149275 eV/atom
Cr5Ti5V115_Rel_Start_17 - RMSD: 2.4297483510263294, RMSE Energy: 0.03092494456149275, RMSE Forces: 0.0031020863733783747, RMSE Stress: 0.0002021366179134847
Atoms 1 energy: -8.907229281532258 eV/atom, Atoms 2 energy: -8.93795677923387 eV/atom, RMSE Energy: 0.030727497701612094 eV/atom
Cr5Ti5V115_Rel_End_23 - RMSD: 1.650411980151367, RMSE Energy: 0.030727497701612094, RMSE Forces: 0.002434709120137479, RMSE Stress: 0.0002797219415533486
Atoms 1 energy: -8.978872205433072 eV/atom, Atoms 2 energy: -9.010251983882874 eV/atom, RMSE Energy: 0.031379778449801776 eV/atom
Cr3Ti5V115W3Zr2_End_Index_66 - RMSD: 1.8556333540024628, RMSE Energy: 0.031379778449801776, RMSE Forces: 0.0013336748492029722, RMSE Stress: 0.00045817100422401594
Atoms 1 energy: -8.971508813149606 eV/atom, Atoms 2 energy: -9.002682663324311 eV/atom, RMSE Energy: 0.031173850174704754 eV/atom
Cr3Ti5V

## From MACE starting point

In [65]:
# vacancy data
vacancy_path = '../data/post_mrs_static/mrs_vac_static'

# get the paths for every OUTCAR file present in the vacancy_path, needs to search recursively
outcar_paths = glob.glob(os.path.join(vacancy_path, '**', 'OUTCAR'), recursive=True)

vac_atoms = {}
for outcar_path in outcar_paths:
    atoms = read(outcar_path)
    # get the name of the final subdirectory
    path_parts = outcar_path.split(os.sep)
    subdir_name = f"{path_parts[-3]}_{path_parts[-2]}"
    vac_atoms[subdir_name] = atoms

print(vac_atoms.keys())

dict_keys(['Cr5Ti5V115_Start_Index_17', 'Cr5Ti5V115_End_Index_23', 'Cr3Ti5V115W3Zr2_End_Index_66', 'Cr3Ti5V115W3Zr2_Start_Index_60', 'Cr4Ti7V111W4Zr2_Start_Index_14', 'Cr4Ti7V111W4Zr2_End_Index_15', 'Cr2Ti2V120W2Zr2_Start_Index_98', 'Cr2Ti2V120W2Zr2_End_Index_117', 'Cr5Ti9V107W4Zr3_Start_Index_34', 'Cr5Ti9V107W4Zr3_End_Index_110', 'Cr6Ti11V102W6Zr3_End_Index_54', 'Cr6Ti11V102W6Zr3_Start_Index_60'])


In [63]:
model_path = '../potentials/gen_6_model_0_L0_isolated-2026-01-16-finetuned_fp64_nh10000_lr1e-4_stagetwo.model'
#model_path = '../potentials/gen_5_model_0-11-28_stagetwo.model'
calc = MACECalculator(model_path=[model_path], device='cuda', use_cueq=True, default_dtype='float32')


def relax_atoms(atoms, calc, relax_cell=False, fmax=0.01):
    """Relax the atoms with the MACE calculator"""
    new_atoms = atoms.copy()
    new_atoms.calc = calc
    if relax_cell:
        fcf = FrechetCellFilter(new_atoms)
        opt = FIRE(fcf)
    else:
        opt = FIRE(new_atoms)
    opt.run(steps=100, fmax=fmax)

    return new_atoms


Default dtype float32 does not match model dtype float64, converting models to float32.


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

In [103]:
in_vac_path = '../data/pre_mrs_comparison/vac/'

# Find all subdirectories recursively within initial_path
subdir_paths = glob.glob(os.path.join(in_vac_path, '**/POSCAR'), recursive=True)
vac_poscar_atoms = {}

for poscar_path in subdir_paths:
    # Get the directory containing POSCAR
    subdir_path = os.path.dirname(poscar_path)
    
    # Get both the composition and index directories
    comp_dir = os.path.basename(os.path.dirname(subdir_path))  # e.g., "Cr4Ti7V111W4Zr2"
    index_dir = os.path.basename(subdir_path)  # e.g., "Start_Index_14"
    
    # Create a meaningful key for the dictionary
    subdir_name = f"{comp_dir}_{index_dir}"
    
    atoms = read(poscar_path)
    print(f"Reading POSCAR from: {subdir_path}")
    vac_poscar_atoms[subdir_name] = atoms

print("\nAtoms objects read from POSCAR files:")
for name in vac_poscar_atoms:
    print(name)

Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr5Ti5V115/Start_Index_17
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr5Ti5V115/End_Index_23
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr3Ti5V115W3Zr2/End_Index_66
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr3Ti5V115W3Zr2/Start_Index_60
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr4Ti7V111W4Zr2/Start_Index_14
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr4Ti7V111W4Zr2/End_Index_15
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr2Ti2V120W2Zr2/Start_Index_98
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr2Ti2V120W2Zr2/End_Index_117
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr5Ti9V107W4Zr3/Start_Index_34
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr5Ti9V107W4Zr3/End_Index_110
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr6Ti11V102W6Zr3/End_Index_54
Reading POSCAR from: ../data/pre_mrs_comparison/vac/Cr6Ti11V102W6Zr3/Start_Index_60

Atoms objects read fr

In [104]:
vac_poscar_atoms['Cr6Ti11V102W6Zr3_End_Index_54'].get_chemical_formula()

'Cr6Ti11V101W6Zr3'

In [105]:
from copy import deepcopy

# First create a mapping of start/end pairs
vacancy_pairs = {}
for vac_key in vac_atoms.keys():
    parts = vac_key.split('_')
    for i, part in enumerate(parts):
        if part in ['Start', 'End']:
            comp_name = '_'.join(parts[:i])
            index_type = part
            index = int(parts[i+2])
            break
    
    # Initialize the dictionary for this composition if it doesn't exist
    if comp_name not in vacancy_pairs:
        vacancy_pairs[comp_name] = {'start_index': None, 'end_index': None}
    
    # Store the indices
    if index_type == 'Start':
        vacancy_pairs[comp_name]['start_index'] = index
    else:  # End
        vacancy_pairs[comp_name]['end_index'] = index

# Now create both vacancies for each composition
vacancy_structures = {}
for comp_name, indices in vacancy_pairs.items():
    start_index = indices['start_index']
    end_index = indices['end_index']
    
    if start_index is None or end_index is None:
        print(f"Warning: Missing start or end index for {comp_name}, skipping...")
        continue
        
    perfect_structure = relaxed_atoms[comp_name]
    
    # Store the position of the start index before deletion
    start_position = perfect_structure[start_index].position.copy()
    
    # Create start vacancy - make fresh copy of perfect structure
    start_key = f"{comp_name}_Start_Index_{start_index}"
    start_structure = deepcopy(perfect_structure)
    del start_structure[start_index]
    
    # Create end vacancy - make fresh copy of perfect structure
    end_key = f"{comp_name}_End_Index_{end_index}"
    end_structure = deepcopy(perfect_structure)
    del end_structure[start_index]  # Remove the atom from its original position
    end_structure[end_index].position = start_position  # Move end atom to start position
    
    # Verify and store start structure
    if start_key in vac_poscar_atoms:
        poscar_comp = vac_poscar_atoms[start_key].get_chemical_formula()
        start_comp = start_structure.get_chemical_formula()
        if poscar_comp == start_comp:
            vacancy_structures[start_key] = relax_atoms(start_structure, calc, relax_cell=False)
            print(f"Successfully created and relaxed vacancy for {start_key}")
        else:
            print(f"Warning: Composition mismatch for {start_key}")
            print(f"Expected: {poscar_comp}, Got: {start_comp}")
    
    # Verify and store end structure
    if end_key in vac_poscar_atoms:
        poscar_comp = vac_poscar_atoms[end_key].get_chemical_formula()
        end_comp = end_structure.get_chemical_formula()
        if poscar_comp == end_comp:
            vacancy_structures[end_key] = relax_atoms(end_structure, calc, relax_cell=False)
            print(f"Successfully created and relaxed vacancy for {end_key}")
        else:
            print(f"Warning: Composition mismatch for {end_key}")
            print(f"Expected: {poscar_comp}, Got: {end_comp}")

print("\nCreated vacancy structures:")
for name in vacancy_structures:
    print(name)

      Step     Time          Energy          fmax
FIRE:    0 23:20:56    -1107.933228        0.482501
FIRE:    1 23:20:56    -1107.959595        0.445611
FIRE:    2 23:20:56    -1108.006592        0.380227
FIRE:    3 23:20:56    -1108.063354        0.341446
FIRE:    4 23:20:56    -1108.121094        0.322119
FIRE:    5 23:20:56    -1108.174194        0.301467
FIRE:    6 23:20:56    -1108.218384        0.277950
FIRE:    7 23:20:56    -1108.252197        0.249383
FIRE:    8 23:20:56    -1108.278809        0.210603
FIRE:    9 23:20:56    -1108.297974        0.208660
FIRE:   10 23:20:56    -1108.305420        0.227045
FIRE:   11 23:20:56    -1108.307251        0.216522
FIRE:   12 23:20:57    -1108.311401        0.196580
FIRE:   13 23:20:57    -1108.316284        0.169152
FIRE:   14 23:20:57    -1108.321533        0.136721
FIRE:   15 23:20:57    -1108.326904        0.101875
FIRE:   16 23:20:57    -1108.331055        0.066984
FIRE:   17 23:20:57    -1108.334595        0.057312
FIRE:   18 23:

In [106]:
# compare the vasp and the mace relaxed atoms 
atoms_names = vacancy_structures.keys()

for atoms_name in atoms_names:
    vasp_atoms = vac_atoms[atoms_name]
    mace_atoms = vacancy_structures[atoms_name]
    rmsd, rmse_energy, rmse_forces, rmse_stress = compare_atoms(vasp_atoms, mace_atoms, verbose=True)
    print(f"{atoms_name} - RMSD: {rmsd}, RMSE Energy: {rmse_energy}, RMSE Forces: {rmse_forces}, RMSE Stress: {rmse_stress}")

Atoms 1 energy: -8.907411827741935 eV/atom, Atoms 2 energy: -8.938239312941029 eV/atom, RMSE Energy: 0.03082748519909373 eV/atom
Cr5Ti5V115_Start_Index_17 - RMSD: 3.6078295210292928, RMSE Energy: 0.03082748519909373, RMSE Forces: 0.002887094809807507, RMSE Stress: 0.0008682447917094323
Atoms 1 energy: -8.907229281532258 eV/atom, Atoms 2 energy: -8.937868179813508 eV/atom, RMSE Energy: 0.030638898281249993 eV/atom
Cr5Ti5V115_End_Index_23 - RMSD: 3.6702702700337033, RMSE Energy: 0.030638898281249993, RMSE Forces: 0.002438264972235236, RMSE Stress: 0.0008230942223744671
Atoms 1 energy: -8.971508813149606 eV/atom, Atoms 2 energy: -9.002650944266732 eV/atom, RMSE Energy: 0.03114213111712516 eV/atom
Cr3Ti5V115W3Zr2_Start_Index_60 - RMSD: 1.2371497356677839, RMSE Energy: 0.03114213111712516, RMSE Forces: 0.00182803030937104, RMSE Stress: 0.0007563213893279876
Atoms 1 energy: -8.978872205433072 eV/atom, Atoms 2 energy: -9.010192390501969 eV/atom, RMSE Energy: 0.03132018506889622 eV/atom
Cr3Ti5

# Compare the NEB VASP and MACE jobs

## From VASP starting point

In [40]:
# neb data
neb_path = '../data/post_mrs_static/mrs_neb_static'

# get the paths for every OUTCAR file present in the neb_path, needs to search recursively
outcar_paths = glob.glob(os.path.join(neb_path, '**', 'OUTCAR'), recursive=True)

neb_atoms = {}
for outcar_path in outcar_paths:
    atoms = read(outcar_path)
    # get the path parts
    path_parts = outcar_path.split(os.sep)
    neb_name = path_parts[-3]  # e.g., "Cr5Ti5V115_17_to_23"
    image_dir = path_parts[-2]  # e.g., "00", "01", etc.
    
    # Initialize the nested dictionary if this is the first image for this NEB
    if neb_name not in neb_atoms:
        neb_atoms[neb_name] = {}
    
    # Add the atoms object to the nested dictionary
    neb_atoms[neb_name][image_dir] = atoms

# Print the structure (optional)
for neb_name, images in neb_atoms.items():
    print(f"\nNEB job: {neb_name}")
    print(f"Number of images: {len(images)}")
    print(f"Image directories: {sorted(images.keys())}")


NEB job: Cr6Ti11V102W6Zr3_60_to_54
Number of images: 7
Image directories: ['00', '01', '02', '03', '04', '05', '06']

NEB job: Cr4Ti7V111W4Zr2_14_to_15
Number of images: 7
Image directories: ['00', '01', '02', '03', '04', '05', '06']

NEB job: Cr2Ti2V120W2Zr2_98_to_117
Number of images: 7
Image directories: ['00', '01', '02', '03', '04', '05', '06']

NEB job: Cr5Ti5V115_17_to_23
Number of images: 7
Image directories: ['00', '01', '02', '03', '04', '05', '06']


In [47]:
from ase.mep import DyNEB 
from ase.optimize import FIRE

model_path = '../potentials/gen_6_model_0_L0_isolated-2026-01-16-finetuned_fp64_nh10000_lr1e-4_stagetwo.model'
#model_path = '../potentials/gen_5_model_0-11-28_stagetwo.model'
#calc = MACECalculator(model_path=[model_path], device='cuda', use_cueq=True, default_dtype='float32')


def neb_relax_atoms(atoms_list, model_path, relax_cell=False, fmax=0.01):
    """Relax the atoms with the MACE calculator"""
    new_atoms_list = [atoms.copy() for atoms in atoms_list]
    for image in new_atoms_list:
        image.calc = MACECalculator(model_path=[model_path], device='cuda', default_dtype='float32')

    neb = DyNEB(new_atoms_list, climb=True)
    neb.interpolate(mic=True)
    opt = FIRE(neb)
    opt.run(steps=100, fmax=0.03)

    return neb


In [45]:
# Load initial POSCAR structures
in_neb_path = '../data/pre_mrs_comparison/neb/'
neb_poscar_atoms = {}

# Find all directories that might contain POSCAR files
dir_paths = glob.glob(os.path.join(in_neb_path, '**/[0-9][0-9]'), recursive=True)

for dir_path in dir_paths:
    # Get path parts
    path_parts = dir_path.split(os.sep)
    neb_name = path_parts[-2]  # e.g., "Cr5Ti5V115_17_to_23"
    image_dir = path_parts[-1]  # e.g., "00", "01", etc.
    
    # Initialize nested dictionary if needed
    if neb_name not in neb_poscar_atoms:
        neb_poscar_atoms[neb_name] = {}
    
    # Check for POSCAR-1 first, then fall back to POSCAR
    poscar_1_path = os.path.join(dir_path, 'POSCAR-1')
    poscar_path = os.path.join(dir_path, 'POSCAR')
    
    if os.path.exists(poscar_1_path):
        atoms = read(poscar_1_path)
        print(f"Reading POSCAR-1 from: {dir_path}")
    elif os.path.exists(poscar_path):
        atoms = read(poscar_path)
        print(f"Reading POSCAR from: {dir_path}")
    else:
        print(f"No POSCAR or POSCAR-1 found in: {dir_path}")
        continue
    
    neb_poscar_atoms[neb_name][image_dir] = atoms

print("\nNEB paths found:")
for neb_name, images in neb_poscar_atoms.items():
    print(f"{neb_name}: {len(images)} images, directories: {sorted(images.keys())}")

Reading POSCAR-1 from: ../data/pre_mrs_comparison/neb/Cr6Ti11V102W6Zr3_60_to_54/03
Reading POSCAR-1 from: ../data/pre_mrs_comparison/neb/Cr6Ti11V102W6Zr3_60_to_54/01
Reading POSCAR-1 from: ../data/pre_mrs_comparison/neb/Cr6Ti11V102W6Zr3_60_to_54/05
Reading POSCAR-1 from: ../data/pre_mrs_comparison/neb/Cr6Ti11V102W6Zr3_60_to_54/02
Reading POSCAR-1 from: ../data/pre_mrs_comparison/neb/Cr6Ti11V102W6Zr3_60_to_54/04
Reading POSCAR from: ../data/pre_mrs_comparison/neb/Cr6Ti11V102W6Zr3_60_to_54/00
Reading POSCAR from: ../data/pre_mrs_comparison/neb/Cr6Ti11V102W6Zr3_60_to_54/06
Reading POSCAR-1 from: ../data/pre_mrs_comparison/neb/Cr4Ti7V111W4Zr2_14_to_15/03
Reading POSCAR-1 from: ../data/pre_mrs_comparison/neb/Cr4Ti7V111W4Zr2_14_to_15/01
Reading POSCAR-1 from: ../data/pre_mrs_comparison/neb/Cr4Ti7V111W4Zr2_14_to_15/05
Reading POSCAR-1 from: ../data/pre_mrs_comparison/neb/Cr4Ti7V111W4Zr2_14_to_15/02
Reading POSCAR-1 from: ../data/pre_mrs_comparison/neb/Cr4Ti7V111W4Zr2_14_to_15/04
Reading POSCA

In [50]:
# Run MACE NEB calculations and compare with VASP
results_summary = {}

for neb_name, images_dict in neb_poscar_atoms.items():
    # Get VASP energies for comparison
    vasp_images = neb_atoms[neb_name]
    vasp_e0 = vasp_images['00'].get_potential_energy()
    vasp_energies = [vasp_images[f"{i:02d}"].get_potential_energy() - vasp_e0 for i in range(7)]
    
    # Convert POSCAR dictionary to ordered list of images for MACE
    ordered_images = [images_dict[f"{i:02d}"] for i in range(7)]
    
    # Run MACE NEB calculation
    neb = neb_relax_atoms(ordered_images, model_path)
    
    # Calculate MACE relative energies from the NEB object
    mace_e0 = neb.images[0].get_potential_energy()
    mace_energies = [img.get_potential_energy() - mace_e0 for img in neb.images]
    
    # Store results
    results_summary[neb_name] = {
        'vasp_barrier': max(vasp_energies),
        'mace_barrier': max(mace_energies),
        'vasp_energies': vasp_energies,
        'mace_energies': mace_energies
    }

# Print summary after all calculations are done
print("\n=== NEB Calculations Summary ===")
for neb_name, results in results_summary.items():
    print(f"\nNEB Path: {neb_name}")
    print(f"VASP barrier height (eV): {results['vasp_barrier']:.3f}")
    print(f"MACE barrier height (eV): {results['mace_barrier']:.3f}")
    print("\nRelative energies along path (eV):")
    print("Image    VASP     MACE")
    print("-----------------------")
    for i in range(7):
        print(f"{i:02d}       {results['vasp_energies'][i]:.3f}    {results['mace_energies'][i]:.3f}")

/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

Default dtype float32 does not match model dtype float64, converting models to float32.
Default dtype float32 does not match model dtype float64, converting models to float32.


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

Default dtype float32 does not match model dtype float64, converting models to float32.
Default dtype float32 does not match model dtype float64, converting models to float32.


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

...

Default dtype float32 does not match model dtype float64, converting models to float32.
Default dtype float32 does not match model dtype float64, converting models to float32.
Default dtype float32 does not match model dtype float64, converting models to float32.


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

Default dtype float32 does not match model dtype float64, converting models to float32.
Default dtype float32 does not match model dtype float64, converting models to float32.
Default dtype float32 does not match model dtype float64, converting models to float32.


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

Default dtype float32 does not match model dtype float64, converting models to float32.
      Step     Time          Energy          fmax
FIRE:    0 22:06:14    -1082.327515       57.861903
FIRE:    1 22:06:14    -1092.699585       29.942585
FIRE:    2 22:06:15    -1097.967896       16.558370
FIRE:    3 22:06:15    -1100.651611        9.857258
FIRE:    4 22:06:15    -1101.881592        6.616178
FIRE:    5 22:06:16    -1102.399658        4.233103
FIRE:    6 22:06:16    -1102.599487        2.581668
FIRE:    7 22:06:16    -1102.660400        2.127416
FIRE:    8 22:06:16    -1102.678711        1.904685
FIRE:    9 22:06:17    -1102.709351        2.339426
FIRE:   10 22:06:17    -1102.776245        2.526832
FIRE:   11 22:06:17    -1102.870483        2.495367
FIRE:   12 22:06:17    -1102.963257        2.509513
FIRE:   13 22:06:18    -1103.057861        2.167969
FIRE:   14 22:06:18    -1103.214111        1.722399
FIRE:   15 22:06:18    -1103.381104        1.230659
FIRE:   16 22:06:18    -1103.5

## From MACE starting point

In [107]:
# neb data
neb_path = '../data/post_mrs_static/mrs_neb_static'

# get the paths for every OUTCAR file present in the neb_path, needs to search recursively
outcar_paths = glob.glob(os.path.join(neb_path, '**', 'OUTCAR'), recursive=True)

neb_atoms = {}
for outcar_path in outcar_paths:
    atoms = read(outcar_path)
    # get the path parts
    path_parts = outcar_path.split(os.sep)
    neb_name = path_parts[-3]  # e.g., "Cr5Ti5V115_17_to_23"
    image_dir = path_parts[-2]  # e.g., "00", "01", etc.
    
    # Initialize the nested dictionary if this is the first image for this NEB
    if neb_name not in neb_atoms:
        neb_atoms[neb_name] = {}
    
    # Add the atoms object to the nested dictionary
    neb_atoms[neb_name][image_dir] = atoms

# Print the structure (optional)
for neb_name, images in neb_atoms.items():
    print(f"\nNEB job: {neb_name}")
    print(f"Number of images: {len(images)}")
    print(f"Image directories: {sorted(images.keys())}")


NEB job: Cr6Ti11V102W6Zr3_60_to_54
Number of images: 7
Image directories: ['00', '01', '02', '03', '04', '05', '06']

NEB job: Cr4Ti7V111W4Zr2_14_to_15
Number of images: 7
Image directories: ['00', '01', '02', '03', '04', '05', '06']

NEB job: Cr2Ti2V120W2Zr2_98_to_117
Number of images: 7
Image directories: ['00', '01', '02', '03', '04', '05', '06']

NEB job: Cr5Ti5V115_17_to_23
Number of images: 7
Image directories: ['00', '01', '02', '03', '04', '05', '06']


In [112]:
from ase.mep import DyNEB 
from ase.optimize import FIRE
from ase.io import write

model_path = '../potentials/gen_6_model_0_L0_isolated-2026-01-16-finetuned_fp64_nh10000_lr1e-4_stagetwo.model'
#model_path = '../potentials/gen_5_model_0-11-28_stagetwo.model'
#calc = MACECalculator(model_path=[model_path], device='cuda', use_cueq=True, default_dtype='float32')


def neb_mace_relax_atoms(start_atoms,end_atoms, model_path, fmax=0.03, save_path=None):
    """Relax the atoms with the MACE calculator"""
    #new_atoms_list = [atoms.copy() for atoms in atoms_list]
    new_atoms_list = [start_atoms.copy() for _ in range(6)]
    new_atoms_list.append(end_atoms.copy())
    print(f"Length of new atoms list: {len(new_atoms_list)}")

    neb = DyNEB(new_atoms_list, climb=True)
    neb.interpolate(mic=True)

    if save_path:
        write(save_path, neb.images)

    for image in neb.images:
        image.calc = MACECalculator(model_paths=[model_path], device='cuda', default_dtype='float32')

    opt = FIRE(neb)
    opt.run(steps=250, fmax=fmax)

    return neb


In [101]:
vacancy_structures.keys()

dict_keys(['Cr5Ti5V115_Start_Index_17', 'Cr5Ti5V115_End_Index_23', 'Cr3Ti5V115W3Zr2_Start_Index_60', 'Cr3Ti5V115W3Zr2_End_Index_66', 'Cr4Ti7V111W4Zr2_Start_Index_14', 'Cr4Ti7V111W4Zr2_End_Index_15', 'Cr2Ti2V120W2Zr2_Start_Index_98', 'Cr2Ti2V120W2Zr2_End_Index_117', 'Cr5Ti9V107W4Zr3_Start_Index_34', 'Cr5Ti9V107W4Zr3_End_Index_110', 'Cr6Ti11V102W6Zr3_Start_Index_60', 'Cr6Ti11V102W6Zr3_End_Index_54'])

In [113]:
# First, let's organize our vacancy structures into pairs
neb_pairs = {}

for vac_key in vacancy_structures.keys():
    # Split the key to get composition and index type
    parts = vac_key.split('_')
    for i, part in enumerate(parts):
        if part in ['Start', 'End']:
            comp_name = '_'.join(parts[:i])
            index_type = part
            index = parts[i+2]  # Keep as string
            break
    
    # Initialize the dictionary for this composition if it doesn't exist
    if comp_name not in neb_pairs:
        neb_pairs[comp_name] = {'start': {'struct': None, 'index': None}, 
                               'end': {'struct': None, 'index': None}}
    
    # Add the structure to the appropriate slot
    if index_type == 'Start':
        neb_pairs[comp_name]['start']['struct'] = vacancy_structures[vac_key]
        neb_pairs[comp_name]['start']['index'] = index
    else:  # End
        neb_pairs[comp_name]['end']['struct'] = vacancy_structures[vac_key]
        neb_pairs[comp_name]['end']['index'] = index

import os
from ase.io import write

# Create output directory if it doesn't exist
save_dir = "../data/test_neb_paths/"
os.makedirs(save_dir, exist_ok=True)

# Run MACE NEB calculations and compare with VASP
results_summary = {}

for comp_name, pair in neb_pairs.items():
    if None in [pair['start']['struct'], pair['end']['struct']]:
        print(f"\nSkipping {comp_name} - missing start or end structure")
        continue
        
    # Create the NEB job name format
    start_idx = pair['start']['index']
    end_idx = pair['end']['index']
    neb_key = f"{comp_name}_{start_idx}_to_{end_idx}"
    
    print(f"\nProcessing NEB for {neb_key}")
    
    # Create save path for initial interpolated path
    save_path = os.path.join(save_dir, f"{neb_key}_initial.xyz")
    
    # Run MACE NEB calculation with save_path
    neb = neb_mace_relax_atoms(pair['start']['struct'], pair['end']['struct'], 
                              model_path, fmax=0.03, save_path=save_path)
    
    try: 
        # Get VASP energies for comparison
        vasp_images = neb_atoms[neb_key]
        vasp_e0 = vasp_images['00'].get_potential_energy()
        vasp_energies = [vasp_images[f"{i:02d}"].get_potential_energy() - vasp_e0 for i in range(7)]
    except:
        print(f"No VASP data found for {neb_key}")
        continue
    
    # Calculate MACE relative energies from the NEB object
    mace_e0 = neb.images[0].get_potential_energy()
    mace_energies = [img.get_potential_energy() - mace_e0 for img in neb.images]
    
    # Store results
    results_summary[neb_key] = {
        'vasp_barrier': max(vasp_energies),
        'mace_barrier': max(mace_energies),
        'vasp_energies': vasp_energies,
        'mace_energies': mace_energies
    }

# Print summary
print("\n=== NEB Calculations Summary ===")
for neb_key, results in results_summary.items():
    print(f"\nNEB Path: {neb_key}")
    print(f"VASP barrier height (eV): {results['vasp_barrier']:.3f}")
    print(f"MACE barrier height (eV): {results['mace_barrier']:.3f}")
    print("\nRelative energies along path (eV):")
    print("Image    VASP     MACE")
    print("-----------------------")
    for i in range(7):
        print(f"{i:02d}       {results['vasp_energies'][i]:.3f}    {results['mace_energies'][i]:.3f}")


Processing NEB for Cr5Ti5V115_17_to_23
Length of new atoms list: 7
Default dtype float32 does not match model dtype float64, converting models to float32.


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

Default dtype float32 does not match model dtype float64, converting models to float32.
Default dtype float32 does not match model dtype float64, converting models to float32.


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

Default dtype float32 does not match model dtype float64, converting models to float32.
Default dtype float32 does not match model dtype float64, converting models to float32.


...

Default dtype float32 does not match model dtype float64, converting models to float32.
Default dtype float32 does not match model dtype float64, converting models to float32.
Default dtype float32 does not match model dtype float64, converting models to float32.


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

Default dtype float32 does not match model dtype float64, converting models to float32.
Default dtype float32 does not match model dtype float64, converting models to float32.


/home/myless/.miniforge3/envs/forge/lib/python3.11/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_loc

      Step     Time          Energy          fmax
FIRE:    0 23:25:38    -1147.454102        1.030014
FIRE:    1 23:25:38    -1147.506226        0.813483
FIRE:    2 23:25:39    -1147.578125        0.570960
FIRE:    3 23:25:39    -1147.634277        0.353765
FIRE:    4 23:25:39    -1147.662598        0.260457
FIRE:    5 23:25:40    -1147.669556        0.431651
FIRE:    6 23:25:40    -1147.669067        0.471411
FIRE:    7 23:25:40    -1147.670532        0.455132
FIRE:    8 23:25:40    -1147.674316        0.426124
FIRE:    9 23:25:40    -1147.678833        0.384166
FIRE:   10 23:25:41    -1147.684326        0.330948
FIRE:   11 23:25:41    -1147.689819        0.268440
FIRE:   12 23:25:41    -1147.695557        0.198819
FIRE:   13 23:25:41    -1147.700317        0.124622
FIRE:   14 23:25:41    -1147.705078        0.081228
FIRE:   15 23:25:42    -1147.708374        0.098416
FIRE:   16 23:25:42    -1147.710449        0.164239
FIRE:   17 23:25:42    -1147.712036        0.217008
FIRE:   18 23:

In [ ]:
# Load initial POSCAR structures
in_neb_path = '../data/pre_mrs_comparison/neb/'
neb_poscar_atoms = {}

# Find all directories that might contain POSCAR files
dir_paths = glob.glob(os.path.join(in_neb_path, '**/[0-9][0-9]'), recursive=True)

for dir_path in dir_paths:
    # Get path parts
    path_parts = dir_path.split(os.sep)
    neb_name = path_parts[-2]  # e.g., "Cr5Ti5V115_17_to_23"
    image_dir = path_parts[-1]  # e.g., "00", "01", etc.
    
    # Initialize nested dictionary if needed
    if neb_name not in neb_poscar_atoms:
        neb_poscar_atoms[neb_name] = {}
    
    # Check for POSCAR-1 first, then fall back to POSCAR
    poscar_1_path = os.path.join(dir_path, 'POSCAR-1')
    poscar_path = os.path.join(dir_path, 'POSCAR')
    
    if os.path.exists(poscar_1_path):
        atoms = read(poscar_1_path)
        print(f"Reading POSCAR-1 from: {dir_path}")
    elif os.path.exists(poscar_path):
        atoms = read(poscar_path)
        print(f"Reading POSCAR from: {dir_path}")
    else:
        print(f"No POSCAR or POSCAR-1 found in: {dir_path}")
        continue
    
    neb_poscar_atoms[neb_name][image_dir] = atoms

print("\nNEB paths found:")
for neb_name, images in neb_poscar_atoms.items():
    print(f"{neb_name}: {len(images)} images, directories: {sorted(images.keys())}")

In [ ]:
# Run MACE NEB calculations and compare with VASP
results_summary = {}

for neb_name, images_dict in neb_poscar_atoms.items():
    # Get VASP energies for comparison
    vasp_images = neb_atoms[neb_name]
    vasp_e0 = vasp_images['00'].get_potential_energy()
    vasp_energies = [vasp_images[f"{i:02d}"].get_potential_energy() - vasp_e0 for i in range(7)]
    
    # Convert POSCAR dictionary to ordered list of images for MACE
    ordered_images = [images_dict[f"{i:02d}"] for i in range(7)]
    
    # Run MACE NEB calculation
    neb = neb_relax_atoms(ordered_images, model_path)
    
    # Calculate MACE relative energies from the NEB object
    mace_e0 = neb.images[0].get_potential_energy()
    mace_energies = [img.get_potential_energy() - mace_e0 for img in neb.images]
    
    # Store results
    results_summary[neb_name] = {
        'vasp_barrier': max(vasp_energies),
        'mace_barrier': max(mace_energies),
        'vasp_energies': vasp_energies,
        'mace_energies': mace_energies
    }

# Print summary after all calculations are done
print("\n=== NEB Calculations Summary ===")
for neb_name, results in results_summary.items():
    print(f"\nNEB Path: {neb_name}")
    print(f"VASP barrier height (eV): {results['vasp_barrier']:.3f}")
    print(f"MACE barrier height (eV): {results['mace_barrier']:.3f}")
    print("\nRelative energies along path (eV):")
    print("Image    VASP     MACE")
    print("-----------------------")
    for i in range(7):
        print(f"{i:02d}       {results['vasp_energies'][i]:.3f}    {results['mace_energies'][i]:.3f}")